In [1]:
import operator
from typing import Tuple, Callable
import sympy as sp
from numpy.random import choice
from sympy import latex
from sympy.abc import x

In [2]:
%run document_generator.ipynb

In [3]:
def random_float_coef():
    return sp.S(choice(list(set(range(-3, 4)) - {0}))) / choice([1, 2, 3])


def func_generator_decorator(expr: sp.Expr) -> Callable[[], sp.Expr]:
    def generate_func() -> sp.Expr:
        symbol = list(expr.free_symbols)[0]
        return random_float_coef() * expr.subs(symbol, random_float_coef() * symbol)

    return generate_func


generate_atan = func_generator_decorator(sp.atan(x))


def generate_raising_to_negative_int() -> sp.Expr:
    return 1 / (random_float_coef() * x + random_float_coef())


def generate_polynom() -> sp.Expr:
    return sum([
        sp.S(choice(list(range(-3, 4)) + i * [0])) / choice([1, 2, 3]) * x ** i for i in range(3)
    ])


def generate_exponentiation() -> sp.Expr:
    return random_float_coef() * sp.exp(choice([-1, 1]) * x)


def generate_random_func():
    operators = [operator.add, operator.mul]
    functions = [generate_raising_to_negative_int, generate_polynom, generate_exponentiation, generate_atan]
    func = None
    while func is None or func.is_constant() or len(latex(func)) < 20:
        random_operator = choice(operators)
        func = random_operator(*map(lambda generator: generator(), choice(functions, size=2, replace=False)))
    return func

In [4]:
def sign_of_func_on_the_sides_of_point(func: sp.Expr,
                                       point: sp.Expr,
                                       left_interval: sp.Interval,
                                       right_interval: sp.Interval) -> Tuple[str, str]:
    set_where_func_not_defined = sp.Reals - sp.calculus.util.continuous_domain(func, x, sp.Reals)

    left_intersection = left_interval.intersect(set_where_func_not_defined)
    if left_intersection is sp.EmptySet:
        if left_interval.inf == -sp.oo:
            left_point = point - 1
        else:
            left_point = (left_interval.inf + point) / 2
    else:
        left_point = (left_intersection.sup + point) / 2

    right_intersection = right_interval.intersect(set_where_func_not_defined)
    if right_intersection is sp.EmptySet:
        if right_interval.sup == sp.oo:
            right_point = point + 1
        else:
            right_point = (right_interval.sup + point) / 2
    else:
        right_point = (right_intersection.inf + point) / 2

    return '+' if func.subs(x, left_point) > 0 else '-', '+' if func.subs(x, right_point) > 0 else '-'


def solveset_plus_solve(func: sp.Expr) -> list[sp.Expr]:
    solveset_roots = sp.solveset(func, x, domain=sp.Reals)
    if isinstance(solveset_roots, sp.FiniteSet):
        return sorted(solveset_roots.args)
    return sorted(filter(lambda root: root.is_real, sp.solve(func, x)))


def find_local_minimums(func: sp.Expr) -> list[sp.Expr]:
    d_func = func.diff(x)
    potential_extremums = solveset_plus_solve(d_func)
    if any(map(lambda point: "LambertW" in str(point), potential_extremums)):
        raise Exception("В решении есть функция LambertW")
    if len(potential_extremums) == 0:
        return []
    potential_extremums_plus_infs = [-sp.oo] + potential_extremums + [sp.oo]
    result = []
    for i in range(1, len(potential_extremums_plus_infs) - 1):
        point = potential_extremums_plus_infs[i]
        interval_left_border = potential_extremums_plus_infs[i - 1]
        interval_right_border = potential_extremums_plus_infs[i + 1]
        if sign_of_func_on_the_sides_of_point(d_func,
                                              point,
                                              sp.Interval(interval_left_border, point),
                                              sp.Interval(point, interval_right_border)) == ('-', '+'):
            result.append(point)
    return result


def find_local_maximums(func: sp.Expr) -> list[sp.Expr]:
    d_func = func.diff(x)
    potential_extremums = solveset_plus_solve(d_func)
    if any(map(lambda point: "LambertW" in str(point), potential_extremums)):
        raise Exception("В решении есть функция LambertW")
    if len(potential_extremums) == 0:
        return []
    potential_extremums_plus_infs = [-sp.oo] + potential_extremums + [sp.oo]
    result = []
    for i in range(1, len(potential_extremums_plus_infs) - 1):
        point = potential_extremums_plus_infs[i]
        interval_left_border = potential_extremums_plus_infs[i - 1]
        interval_right_border = potential_extremums_plus_infs[i + 1]
        if sign_of_func_on_the_sides_of_point(d_func,
                                              point,
                                              sp.Interval(interval_left_border, point),
                                              sp.Interval(point, interval_right_border)) == ('+', '-'):
            result.append(point)
    return result


def find_inflection_points(func: sp.Expr) -> list[sp.Expr]:
    d2_func = func.diff(x, 2)
    potential_inflection_points = solveset_plus_solve(d2_func)
    if any(map(lambda point: "LambertW" in str(point), potential_inflection_points)):
        raise Exception("В решении есть функция LambertW")
    if len(potential_inflection_points) == 0:
        return []
    potential_inflection_points_plus_infs = [-sp.oo] + potential_inflection_points + [sp.oo]
    result = []
    for i in range(1, len(potential_inflection_points_plus_infs) - 1):
        point = potential_inflection_points_plus_infs[i]
        interval_left_border = potential_inflection_points_plus_infs[i - 1]
        interval_right_border = potential_inflection_points_plus_infs[i + 1]
        signs = sign_of_func_on_the_sides_of_point(d2_func,
                                                   point,
                                                   sp.Interval(interval_left_border, point),
                                                   sp.Interval(point, interval_right_border))
        if signs == ('+', '-') or signs == ('-', '+'):
            result.append(point)
    return result


def find_oblique_asymptotes(func: sp.Expr) -> list[sp.Expr]:
    result = set()
    for lim in [+sp.oo, -sp.oo]:
        k = (func / x).limit(x, lim)
        if k != sp.oo and k != -sp.oo:
            b = (func - k * x).limit(x, lim)
            if b != sp.oo and b != -sp.oo:
                result.add(k * x + b)
    return list(result)


def find_vertical_asymptotes(func: sp.Expr) -> list[sp.Expr]:
    set_where_func_not_defined = (sp.Reals - sp.calculus.util.continuous_domain(func, x, sp.Reals)).simplify()
    if not isinstance(set_where_func_not_defined, (sp.FiniteSet, sp.Interval)):
        raise Exception("Не получается проанализировать множество точек, "
                        "в которых функция не существует, поэтому невозможно найти асимптоты")
    result = []

    def is_asymptote(number: sp.Expr) -> bool:
        limit_plus = func.limit(x, set_of_numbers, '+')
        limit_minus = func.limit(x, set_of_numbers, '-')
        if limit_plus == sp.oo or limit_plus == -sp.oo or limit_minus == sp.oo or limit_minus == -sp.oo:
            return True
        return False

    for set_of_numbers in set_where_func_not_defined.args:
        if isinstance(set_of_numbers, sp.Expr):
            if is_asymptote(set_of_numbers):
                result.append(set_of_numbers)
        elif isinstance(set_of_numbers, sp.FiniteSet):
            for number in set_of_numbers.args:
                if is_asymptote(number):
                    result.append(number)
        else:
            raise Exception("Не получается проанализировать множество точек, "
                            "в которых функция не существует, поэтому невозможно найти асимптоты")
    return result

In [5]:
DESCRIPTION = "Исследовать функцию f(x) с помощью производной, найти необходимые пределы и решить уравнения. Построить график функции и асимптот (если есть), отметить и подписать точки экстремума и точки перегиба (если есть), включить функцию и асимптоты (если есть) в легенду."

In [6]:
def task_condition(func: sp.Expr) -> str:
    return f"\n\n$$f(x) = {latex(func)}$$"

In [7]:
def solve_task(func: sp.Expr) -> str:
    def to_latex(info: list[sp.Expr], lambda_func: Callable[[sp.Expr], str] = latex):
        return f"${', '.join(map(lambda_func, info))}$" if len(info) != 0 else "нет"

    return f"\n\nФункция: $f(x) = {latex(func)}$\n\n" \
           f"Локальные максимумы в точках: {to_latex(find_local_maximums(func))}\n\n" \
           f"Локальные минимумы в точках: {to_latex(find_local_minimums(func))}\n\n" \
           f"Точки перегиба: {to_latex(find_inflection_points(func))}\n\n" \
           f"Вертикальные асимптоты: {to_latex(find_vertical_asymptotes(func), lambda asymptote: f'x = {latex(asymptote)}')}\n\n" \
           f"Наклонные асимптоты: {to_latex(find_oblique_asymptotes(func), lambda asymptote: f'y = {latex(asymptote)}')}"

In [8]:
def generate_task_and_answer(option: int) -> Tuple[str, str]:
    while True:
        try:
            func = generate_random_func().simplify()
            solution = solve_task(func)
            if len(solution) > 300:
                continue
            return task_condition(func), solution
        except:
            pass

In [9]:
doc = DocumentGenerator(generate_task_and_answer, DESCRIPTION)

In [10]:
write_tasks_and_solutions(doc, "../out/tasks/task-6.tex", "../out/answers/answer-6.tex", 150)